In [3]:
import scipy.stats as st
import numpy as np
import pandas as pd
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS


In [78]:
#import lstop data
lstops = pd.read_csv('../data/lstops.csv', delimiter=',', quotechar='"')
#check column
lstops["Location"] = lstops["Location"].str.replace("(","") 
lstops["Location"] = lstops["Location"].str.replace(")","") 
lat_list = []
lon_list = []
for each in lstops["Location"]:
    x = each.split(",")
    lat = float(x[0])
    lon = float(x[1])
    lat_list.append(lat)
    lon_list.append(lon)

lstops["Lat"] = lat_list
lstops["Lon"] = lon_list

lstops

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,Lat,Lon
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"41.857908, -87.669147",41.857908,-87.669147
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"41.857908, -87.669147",41.857908,-87.669147
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"41.829353, -87.680622",41.829353,-87.680622
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"41.829353, -87.680622",41.829353,-87.680622
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"41.831677, -87.625826",41.831677,-87.625826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,30106,S,Wilson (95th-bound),Wilson,Wilson (Red & Purple Lines),40540,True,True,False,False,False,False,False,False,False,False,"41.964273, -87.657588",41.964273,-87.657588
296,30383,N,Washington/Wabash (Outer Loop),Washington/Wabash,"Washington/Wabash (Brown, Green, Orange, Purpl...",41700,True,False,False,True,True,False,False,False,False,False,"41.88322, -87.626189",41.883220,-87.626189
297,30385,S,Wilson (Loop-bound),Wilson,Wilson (Red & Purple Lines),40540,True,False,False,False,False,False,True,False,False,False,"41.964273, -87.657588",41.964273,-87.657588
298,30033,W,Ashland (Harlem-54th/Cermak-bound),Ashland,Ashland (Green & Pink Lines),40170,True,False,False,True,False,False,False,False,True,False,"41.885269, -87.666969",41.885269,-87.666969


In [110]:
ca_to_zip = pd.read_csv('../data/ca_zip.csv',header=None)
ca_to_zip = ca_to_zip.drop(ca_to_zip.index[0])
ca_to_zip.columns = ["Community Area", "Zipcode", "Pop2010"]
ca_to_zip


,Community Area,Zipcode,Pop2010
1,01,60626,48980
2,01,60645,5820
3,01,60660,191
4,02,60626,1159
5,02,60645,39454
...,...,...,...
230,76,60634,60
231,76,60656,12159
232,76,60706,57
233,77,60640,16678


In [80]:
df = lstops

gis = GIS()

def get_zip(df, lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]), "y":float(df[lat_field]), "spatialReference":{"wkid": 4326}})))
    return location['address']['Postal']

        
lstops["zips"] = df.apply(get_zip, axis=1, lat_field='Lat', lon_field='Lon')
lstops["zips"]

#adapted from:
#https://gis.stackexchange.com/questions/352961/convert-lat-lon-to-zip-postal-code-using-python?
#fbclid=IwAR25An6-uDiY1Lpy5EVPUG15RcrRKj5fL1M86Bt3Svqy_iVRi1YLLfviRT0



0      60608
1      60608
2      60609
3      60609
4      60616
       ...  
295    60640
296    60602
297    60640
298    60612
299    60612
Length: 300, dtype: object

In [115]:
lstops["zips"] = zipcodes
lstops.merge(ca_to_zip, left_on="zips", right_on="Zipcode")


,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,...,Y,Pnk,O,Location,Lat,Lon,zips,Community Area,Zipcode,Pop2010
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,True,False,"41.857908, -87.669147",41.857908,-87.669147,60608,28,60608,7667
1,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,True,False,"41.857908, -87.669147",41.857908,-87.669147,60608,29,60608,2283
2,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,True,False,"41.857908, -87.669147",41.857908,-87.669147,60608,30,60608,16654
3,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,True,False,"41.857908, -87.669147",41.857908,-87.669147,60608,31,60608,34380
4,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,True,False,"41.857908, -87.669147",41.857908,-87.669147,60608,59,60608,6594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,30142,S,Washington/Wells (Outer Loop),Washington/Wells,"Washington/Wells (Brown, Orange, Purple & Pink...",40730,True,False,False,False,...,False,False,False,"41.882695, -87.63378",41.882695,-87.633780,60606,32,60606,525
956,30073,S,Washington (Forest Pk-bound),Washington,Washington (Blue Line),40370,False,False,True,False,...,False,False,False,"41.883164, -87.62944",41.883164,-87.629440,60602,32,60602,1204
957,30072,N,Washington (O'Hare-bound),Washington,Washington (Blue Line),40370,False,False,True,False,...,False,False,False,"41.883164, -87.62944",41.883164,-87.629440,60602,32,60602,1204
958,30384,S,Washington/Wabash (Inner Loop),Washington/Wabash,"Washington/Wabash (Brown, Green, Orange, Purpl...",41700,True,False,False,True,...,False,True,True,"41.88322, -87.626189",41.883220,-87.626189,60602,32,60602,1204


In [117]:
lstops.to_csv("lstops_localized.csv",index=False)